# Manipulaciones y ajustes de histogramas

## Librerías de histogramas

Python tiene librerías para llenar histogramas.

### NumPy

NumPy, por ejemplo, tiene una función [np.histogram](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html).

In [ ]:
import skhep_testdata, uproot

tree = uproot.open(skhep_testdata.data_path("uproot-Zmumu.root"))["events"]

import numpy as np

np.histogram(tree["M"].array())

Debido a la prominencia de NumPy, este 2-tupla de arreglos (contenidos de los bins y bordes) es un formato de histograma ampliamente reconocido, aunque carece de muchas de las características que los físicos de alta energía esperan (sub/sobre flujo, etiquetas de ejes, incertidumbres, etc.).

### Matplotlib

Matplotlib también tiene una función [plt.hist](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html).

In [ ]:
import matplotlib.pyplot as plt

plt.hist(tree["M"].array())

Además de los mismos contenidos de bins y bordes que NumPy, Matplotlib incluye un gráfico que se puede graficar.

### Boost-histogram y hist

La característica principal que falta en estas funciones (sin algún esfuerzo) es la posibilidad de rellenar nuevamente. Los físicos de altas energías generalmente quieren llenar histogramas con más datos de los que pueden caber en memoria, lo que significa establecer intervalos de bins en un contenedor vacío y llenarlo en lotes (secuencialmente o en paralelo).

Boost-histogram es una biblioteca diseñada para ese propósito. Está destinada a ser un componente de infraestructura. Puedes explorar su funcionalidad "de bajo nivel" al importarla:

In [ ]:
import boost_histogram as bh

Una capa más amigable para el usuario (con gráficos, por ejemplo) es proporcionada por una biblioteca llamada "hist."

In [ ]:
import hist

h = hist.Hist(hist.axis.Regular(120, 60, 120, name="masa"))

h.fill(tree["M"].array())

h.plot()

### Indexación Universal de Histogramas (UHI)

Hay un intento dentro de Scikit-HEP para estandarizar lo que significan las rebanadas similares a arreglos para un histograma. ([Ver documentación](https://uhi.readthedocs.io/en/latest/indexing.html).)

Naturalmente, las rebanadas enteras deberían seleccionar un rango de bins,

In [ ]:
h[10:110].plot()

pero a menudo deseamos seleccionar bins por valor de coordenada.

In [ ]:
# Versión explícita
h[hist.loc(90) :].plot()

# Versión corta
h[90j:].plot()

o rebinar por un factor,

In [ ]:
# Versión explícita
h[:: hist.rebin(2)].plot()

# Versión corta
h[::2j].plot()

o sumar sobre un rango.

In [ ]:
# Versión explícita
h[hist.loc(80) : hist.loc(100) : sum]

# Versión corta
h[90j:100j:sum]

Las cosas se vuelven más interesantes cuando un histograma tiene múltiples dimensiones.

In [ ]:
import uproot
import hist
import awkward as ak

picodst = uproot.open(
    "https://pivarski-princeton.s3.amazonaws.com/pythia_ppZee_run17emb.picoDst.root:PicoDst"
)

hist_vertices = hist.Hist(
    hist.axis.Regular(600, -1, 1, label="x"),
    hist.axis.Regular(600, -1, 1, label="y"),
    hist.axis.Regular(40, -200, 200, label="z"),
)

datos_vertices = picodst.arrays(filter_name="*mPrimaryVertex[XYZ]")

hist_vertices.fill(
    ak.flatten(datos_vertice["Event.mPrimaryVertexX"]),
    ak.flatten(datos_vertice["Event.mPrimaryVertexY"]),
    ak.flatten(datos_vertice["Event.mPrimaryVertexZ"]),
)

hist_vertices[:, :, sum].plot2d_full()

In [ ]:
hist_vertices[-0.25j:0.25j, -0.25j:0.25j, sum].plot2d_full()

In [ ]:
hist_vertices[sum, sum, :].plot()

In [ ]:
hist_vertices[-0.25j:0.25j:sum, -0.25j:0.25j:sum, :].plot()

Un objeto de histograma puede tener más dimensiones de las que se pueden visualizar razonablemente; puedes rebanar, rebin y proyectar en algo visual más tarde.

## Ajuste de histogramas

Escribiendo directamente una función de pérdida en Minuit:

In [ ]:
import numpy as np
import iminuit.cost

norm = len(h.axes[0].widths) / (h.axes[0].edges[-1] - h.axes[0].edges[0]) / h.sum()


def f(x, background, mu, gamma):
    return (
        background
        + (1 - background) * gamma**2 / ((x - mu) ** 2 + gamma**2) / np.pi / gamma
    )


loss = iminuit.cost.LeastSquares(
    h.axes[0].centers, h.values() * norm, np.sqrt(h.variances()) * norm, f
)
loss.mask = h.variances() > 0

minimizer = iminuit.Minuit(loss, background=0, mu=91, gamma=4)

minimizer.migrad()
minimizer.hesse()

(h * norm).plot()
plt.plot(loss.x, f(loss.x, *minimizer.values))

O a través de zfit, un ajustador similar a RooFit en Python:

In [ ]:
import zfit

binned_data = zfit.data.BinnedData.from_hist(h)

binning = zfit.binned.RegularBinning(120, 60, 120, name="masa")
space = zfit.Space("masa", binning=binning)

background = zfit.Parameter("background", 0)
mu = zfit.Parameter("mu", 91)
gamma = zfit.Parameter("gamma", 4)
unbinned_model = zfit.pdf.SumPDF(
    [zfit.pdf.Uniform(60, 120, space), zfit.pdf.Cauchy(mu, gamma, space)], [background]
)

model = zfit.pdf.BinnedFromUnbinnedPDF(unbinned_model, space)
loss = zfit.loss.BinnedNLL(model, binned_data)

minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(loss)

binned_data.to_hist().plot(density=1)
model.to_hist().plot(density=1)